**Data Wrangling**

First we must manually label a small subset of our data. To do this, we will create a CSV.

In [1]:
# manually create a CSV file to label our data
# (do this in csv file in folder instead)

import csv

simpsons = [
    ['filename', 'character', 'emotion'],
    ['1.png', 'homer_simpson'],
    [],
    []
]


**PCA Code for Image Reduction**

In [ ]:
# import packages
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
# import dataset

In [ ]:

# convert images into a 1D array
flattened_images = images.reshape(image.shape[0], -1)

# normalize the pixel values,
#      to have a range of [0, 1],
#      standardize to have mean of 0 and s.d. of 1
scaler = StandardScaler()
images_normalized = scaler.fit_transform(flattened_images)


**PCA Code for Each Class**

**Test: Reconstruct each PCA model, then measure distance between reconstruction and input**

**Find & Discuss Lowest Distance**